In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
!git clone https://github.com/Electromensch/Grease-detect

Cloning into 'Grease-detect'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [9]:
df = pd.read_csv('Grease-detect/vibration_data_and_header.csv')
#df['total'] = X.apply(sum, axis=1)
df.head()
#(df[df['low_grease']==0].total.min(), df[df['low_grease']==0].total.max())


,identifier,low_grease,s1_fb1,s1_fb2,s1_fb3,s1_fb4,s1_fb5,s2_fb1,s2_fb2,s2_fb3,s2_fb4,s2_fb5
0,1,0,193.354127,199.216030,163.355636,192.992784,191.464309,194.993160,197.404228,179.661069,195.639494,187.282497
1,2,0,189.812360,197.276925,160.930905,197.587089,192.195973,194.666661,201.669789,187.436239,196.766512,191.081678
2,3,0,195.701129,197.603492,160.558204,195.922677,194.815197,194.713178,196.443662,188.786681,197.238927,194.375613
3,4,0,185.222741,195.595038,174.492461,194.051625,198.489453,200.517259,196.386578,200.080633,197.848037,206.754103
4,5,0,194.886473,196.721925,180.923926,193.229550,194.106990,196.399191,190.816571,190.248573,201.052747,206.314634


In [10]:
#df.tail(10)
#(df[df['low_grease']==1].total.min(), df[df['low_grease']==1].total.max() )

In [11]:
X = df.drop(df[['identifier', 'low_grease']], axis=1)
y = df.low_grease

In [12]:
corel = X.corr()
corel['diff'] = abs(corel[corel.columns]).sum(axis=1)
corel['diff'] = (corel['diff'] - 1)/9
corel

,s1_fb1,s1_fb2,s1_fb3,s1_fb4,s1_fb5,s2_fb1,s2_fb2,s2_fb3,s2_fb4,s2_fb5,diff
s1_fb1,1.000000,-0.095896,-0.135694,0.575644,-0.203017,0.916722,-0.149892,-0.271194,0.534567,-0.275239,0.350874
s1_fb2,-0.095896,1.000000,-0.043519,0.055642,0.035878,0.000329,0.910997,0.013109,0.161695,-0.029341,0.149601
s1_fb3,-0.135694,-0.043519,1.000000,-0.178150,0.614246,-0.100592,0.071346,0.560362,-0.153464,0.554315,0.267965
s1_fb4,0.575644,0.055642,-0.178150,1.000000,-0.051482,0.543603,-0.003723,-0.190884,0.915100,-0.124573,0.293200
s1_fb5,-0.203017,0.035878,0.614246,-0.051482,1.000000,-0.172919,0.154276,0.677660,-0.023558,0.910521,0.315951
s2_fb1,0.916722,0.000329,-0.100592,0.543603,-0.172919,1.000000,-0.053908,-0.276654,0.549628,-0.272505,0.320762
s2_fb2,-0.149892,0.910997,0.071346,-0.003723,0.154276,-0.053908,1.000000,0.141032,0.098487,0.091226,0.186098
s2_fb3,-0.271194,0.013109,0.560362,-0.190884,0.677660,-0.276654,0.141032,1.000000,-0.194771,0.720296,0.338440
s2_fb4,0.534567,0.161695,-0.153464,0.915100,-0.023558,0.549628,0.098487,-0.194771,1.000000,-0.126283,0.306395
s2_fb5,-0.275239,-0.029341,0.554315,-0.124573,0.910521,-0.272505,0.091226,0.720296,-0.126283,1.000000,0.344922


In [13]:
corel.iloc[:5, 5:-1]

,s2_fb1,s2_fb2,s2_fb3,s2_fb4,s2_fb5
s1_fb1,0.916722,-0.149892,-0.271194,0.534567,-0.275239
s1_fb2,0.000329,0.910997,0.013109,0.161695,-0.029341
s1_fb3,-0.100592,0.071346,0.560362,-0.153464,0.554315
s1_fb4,0.543603,-0.003723,-0.190884,0.915100,-0.124573
s1_fb5,-0.172919,0.154276,0.677660,-0.023558,0.910521


In [14]:
from sklearn.model_selection import train_test_split
# 1 way to use features
#X = X.iloc[:, :5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score

In [16]:


def call_grd_search(clf, params, X_data, y_data, scoring, cv):
    kfolds = StratifiedKFold(cv)
    gscv = GridSearchCV(clf, param_grid=params, cv=kfolds.split(X_data, y_data), scoring=scoring, n_jobs=-1).fit(X_data, y_data)
    return gscv

def choose_best_clf(classifiers_dict, X_data, y_data, scoring='precision', cv=5):
    best_score, best_clf = 0, None
    for name, clf in classifiers_dict.items():
        clf, params = clf
        clf = Pipeline(steps=[('scaler', StandardScaler()), (name, clf)])
        gscv = call_grd_search(clf, params, X_data, y_data, scoring, cv)
        score = gscv.best_score_
        parametrs =gscv.best_params_
        print(f'Classifier : {name}, mean {scoring}: {score}')
        print(f'Best parameters : {parametrs}')
        if score > best_score:
            best_score = score
            best_clf = (name, gscv.best_estimator_)
    print(f'Best classifier : {best_clf[0]}, mean {scoring}: {best_score}')
    return best_clf




In [17]:
seed = 0

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [19]:
X_new = pd.DataFrame()

gboost = GradientBoostingClassifier(max_depth=12, max_features=2, min_samples_leaf=7,
                                    min_samples_split=20, random_state=seed).fit(X_train, y_train)


In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
mlp = MLPClassifier(activation='relu', hidden_layer_sizes=(128, 64, 16,), alpha=0.001, max_iter=150,
                    random_state=seed).fit(X_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [61]:
svm = SVC(C=500, degree= 1, kernel='rbf', random_state=seed, probability=True).fit(X_train, y_train)


In [62]:
print('Gradient boosting f1 : {}'.format(f1_score(y_true=y_test, y_pred=gboost.predict(X_test))))
print('SVM f1 : {}'.format(f1_score(y_true=y_test, y_pred=svm.predict(X_test))))
print('MLP f1 : {}'.format(f1_score(y_true=y_test, y_pred=mlp.predict(X_test_scaled))))


Gradient boosting f1 : 0.849772382397572
SVM f1 : 0.8436699128651973
MLP f1 : 0.8164300202839755


In [63]:
X_new['xgboost_prob_0'] = gboost.predict_proba(X)[:,0]
X_new['xgboost_prob_1'] = gboost.predict_proba(X)[:,1]

X_new['svm_prob_0'] = svm.predict_proba(X)[:,0]
X_new['svm_prob_1'] = svm.predict_proba(X)[:,1]

X_new['mlp_prob_0'] = mlp.predict_proba(X)[:,0]
X_new['mlp_prob_1'] = mlp.predict_proba(X)[:,1]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [64]:
X_new.shape, y.shape

((30380, 6), (30380,))

In [65]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new, y, test_size=0.1, random_state=0)

In [66]:
from collections import Counter

In [69]:
def majority_voting_average(X:pd.DataFrame, a:float=0.6):
  val0 = (X['xgboost_prob_0']*a+(1-a)*X['svm_prob_0']).values
  val1 = (X['xgboost_prob_1']*a+(1-a)*X['svm_prob_1']).values
  y = np.argmax(list(zip(list(val0), list(val1))), axis=1)
  return y

In [68]:
def majority_voting_count(X):
  y = X.apply(lambda x: max(Counter(x).items(), key=lambda x: x[1])[0], axis=1)
  return y

In [87]:
f1_score(y_true=y_train2, y_pred=majority_voting_average(X_train2, 0.5))

0.9565746528738176

In [86]:
f1_score(y_true=y_test2, y_pred=majority_voting_average(X_test2, 0.5))

0.8568496664956387

In [110]:
lr = SVC(C=0.0007, degree= 1, kernel='rbf', random_state=seed, probability=False)

In [111]:
lr.fit(X_train2, y_train2)

SVC(C=0.0007, degree=1, random_state=0)

In [112]:
f1_score(y_true=y_test2, y_pred=lr.predict(X_test2))

0.8581488933601609